In [69]:
import pandas as pd
from pathlib import Path
from src.metric_utils import load_tranco_list

OUTPUT = "crawl_and_store_eligible_domains.parquet"

YEAR_TO_CRAWL = "2024"
MONTH_TO_CRAWL = "04"

CRAWL_MAX_DEPTH = 40  # resources

In [47]:
available_data_files = list(Path("metric_data/nel_domain_resource_monitoring_stats").glob("*.parquet"))

latest_file = available_data_files[-1]
latest_file

WindowsPath('metric_data/nel_domain_resource_monitoring_stats/2022-02.parquet')

In [52]:
tranco_list = load_tranco_list(YEAR_TO_CRAWL, MONTH_TO_CRAWL)
if tranco_list is None:
    raise Exception(f"Fetch yourself a tranco list for date '{YEAR_TO_CRAWL}-{MONTH_TO_CRAWL}' in order for this to work")

tranco_list

,popular_domain_name
0,google.com
1,facebook.com
2,a-msedge.net
3,amazonaws.com
4,microsoft.com
...,...
4282597,72517.cn
4282598,enterpriseforce.co.uk
4282599,gamepulsepro.store
4282600,tucsonlifestyle.com


In [66]:
data = pd.read_parquet(latest_file)
data['url_domain_hosted_resources_with_nel'] = data['url_domain_hosted_resources_with_nel'].astype("UInt32")
data['url_domain'] = data['url_domain'].astype("object")

popular_domain_data = data[data['url_domain'].isin(tranco_list['popular_domain_name'])]
popular_domain_data

,date,url_domain,url_domain_hosted_resources,url_domain_hosted_resources_with_nel,url_domain_monitored_resources_ratio
15,2022-02,001k.exchange,49,49,100.000000
18,2022-02,001xnxx.com,5,5,100.000000
21,2022-02,002mag.com,33,32,96.970001
23,2022-02,003ms.ru,2,2,100.000000
26,2022-02,0067.ru,3,3,100.000000
...,...,...,...,...,...
972149,2022-02,zztt21.com,8,8,100.000000
972152,2022-02,zztt24.com,16,16,100.000000
972153,2022-02,zztt26.com,8,8,100.000000
972161,2022-02,zzzchan.xyz,8,8,100.000000


In [71]:
eligible_domain_data = popular_domain_data[
    popular_domain_data['url_domain_hosted_resources_with_nel'] < CRAWL_MAX_DEPTH
    ]
eligible_domain_data

,date,url_domain,url_domain_hosted_resources,url_domain_hosted_resources_with_nel,url_domain_monitored_resources_ratio
18,2022-02,001xnxx.com,5,5,100.000000
21,2022-02,002mag.com,33,32,96.970001
23,2022-02,003ms.ru,2,2,100.000000
26,2022-02,0067.ru,3,3,100.000000
33,2022-02,007soccerpicks.com,3,3,100.000000
...,...,...,...,...,...
972149,2022-02,zztt21.com,8,8,100.000000
972152,2022-02,zztt24.com,16,16,100.000000
972153,2022-02,zztt26.com,8,8,100.000000
972161,2022-02,zzzchan.xyz,8,8,100.000000


In [75]:
eligible_domain_data.to_parquet(OUTPUT)